In [2]:
import networkx as nx
import numpy as np

from snpp.utils.matrix import load_sparse_csr
from snpp.cores.correlation_clustering import sampling_wrapper, \
    agglomerative

dataset = 'wiki'

In [3]:
g = nx.from_scipy_sparse_matrix(load_sparse_csr('data/{}.npz'.format(dataset)),
                                create_using=nx.Graph(), edge_attribute='sign')

In [4]:
print('number of nodes {}'.format(g.number_of_nodes()))


number of nodes 11259


In [5]:
sample_size = 2000
threshold = -0.7

In [6]:
clus = sampling_wrapper(g, agglomerative,
                        threshold=threshold,
                        sample_size=sample_size)

0it [00:00, ?it/s]

sample_size 2000


928it [01:49, 61.23it/s]
  0%|          | 27/9259 [00:00<00:34, 264.95it/s]

partition on the samples produces 1073 clusters
remainign nodes to assign clusters 9259


100%|██████████| 9259/9259 [00:42<00:00, 217.54it/s]
0it [00:00, ?it/s]

singleton_nodes (3173)


113it [00:06,  2.16it/s]


{0: {2}, 1: {8}, 2: {2732}, 3: {11}, 4: {8204}, 5: {8206}, 6: {17}, 7: {22}, 8: {24}, 9: {25}, 10: {26}, 11: {8219}, 12: {31}, 13: {8228}, 14: {39}, 15: {40}, 16: {41}, 17: {8235}, 18: {2738}, 19: {8239}, 20: {48}, 21: {50}, 22: {8245}, 23: {55}, 24: {60}, 25: {8253}, 26: {8255}, 27: {64}, 28: {8257}, 29: {8259}, 30: {71}, 31: {8265}, 32: {75}, 33: {78}, 34: {79}, 35: {80}, 36: {8273}, 37: {1379}, 38: {84}, 39: {14}, 40: {86}, 41: {88}, 42: {90}, 43: {8285}, 44: {94}, 45: {8287}, 46: {8288}, 47: {101}, 48: {8294}, 49: {8296}, 50: {105, 11076}, 51: {106}, 52: {107}, 53: {108}, 54: {8301}, 55: {8302}, 56: {8303}, 57: {113}, 58: {116}, 59: {117}, 60: {118}, 61: {8311}, 62: {121}, 63: {6847}, 64: {124}, 65: {126}, 66: {129}, 67: {133}, 68: {134}, 69: {135}, 70: {137}, 71: {8330}, 72: {8331}, 73: {141}, 74: {142}, 75: {144}, 76: {145}, 77: {8339}, 78: {8340, 3367}, 79: {149}, 80: {151}, 81: {8348}, 82: {158}, 83: {8352}, 84: {8353}, 85: {162}, 86: {8358}, 87: {167}, 88: {168}, 89: {169}, 90

In [7]:
len(clus)

4126

In [8]:
sizes = list(map(len, clus.values()))
np.sort(sizes)[-100:]

array([   2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
          3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
          3,    3,    4,    4,    4,    4,    4,    4,    4,    4,    4,
          4,    4,    4,    5,    5,    5,    5,    5,    6,    6,    7,
          8,    9,   10,   12,   13,   15,   22,   46,  117,  283,  334,
       6077])

In [9]:
max_c = max(clus, key=lambda c: len(clus[c]))
print(max_c)

3


In [10]:
[c for c in clus if len(clus[c]) == 2]

[67,
 139,
 221,
 303,
 311,
 320,
 337,
 346,
 367,
 377,
 444,
 493,
 548,
 629,
 631,
 678,
 709,
 713,
 765,
 779,
 783,
 796,
 855,
 868,
 900,
 987,
 994,
 1122,
 1150,
 1177,
 1259,
 1260,
 1272,
 1315,
 1386,
 1388,
 1450,
 1469,
 1506,
 1520,
 1614,
 1685,
 1688,
 1766,
 1787,
 1852,
 1911,
 1961,
 2005,
 2019,
 2057,
 2105,
 2163,
 2164,
 2166,
 2207,
 2228,
 2242,
 2343,
 2377,
 2419,
 2467,
 2513,
 2583,
 2617,
 2618,
 2704,
 2753,
 2919,
 2966,
 2994,
 3052,
 3183,
 3290,
 3389,
 3444,
 3461,
 3513,
 3902]

In [11]:
def sign_percent(g):
    df = pd.DataFrame([(i, j, g[i][j]['sign'])
             for i, j in g.edges_iter()], columns=('u', 'v', 's'))
    return df['s'].value_counts() / df.shape[0]

In [12]:
main_component = g.subgraph(clus[max_c])
sign_percent(main_component)

 1.0    0.871442
-1.0    0.128558
Name: s, dtype: float64

In [13]:
sign_percent(g)

 1.0    0.774805
-1.0    0.225195
Name: s, dtype: float64

In [ ]:
g[227]

In [ ]:
g.number_of_nodes()

In [ ]:
for l, nodes in clus.items():
    for n in nodes:
        g.node[n]['ClusterId'] =    l

In [ ]:
nx.write_gpickle(g, 'output/clus_wiki.pkl')

In [ ]:
g = nx.read_gpickle('output/clus_wiki.pkl')

In [ ]:
g.node[0]

In [ ]:
for n in g.nodes_iter():
    del g.node[n]['cc_label']

In [ ]:
nx.write_gml(g, 'output/clus_wiki.gml')